# Calculating Excess Atom Fraction (EAF)
This script will calculate the EAF for each contig in the viral and cell enriched fractions.

# Import needed packages

In [1]:
import pandas as pd
from sklearn.preprocessing import normalize
import re
from Bio import Entrez
import time
import numpy as np

# Functions

In [4]:
# Function to search NCBI for taxa level
# # Set email for NCBI Entrez API
# Entrez.email = "your_email@example.com"
pd.set_option('display.precision', 5)
def get_taxonomic_level(taxid, retries=3):
    for attempt in range(retries):
        try:
            # Fetch the taxonomy record from NCBI Entrez
            handle = Entrez.efetch(db="taxonomy", id=str(taxid), retmode="xml")
            records = Entrez.read(handle)
            handle.close()
            
            # Extract the rank from the taxonomy record
            taxonomic_level = records[0].get("Rank", "no rank")
            return taxonomic_level
        except Exception as e:
            if attempt < retries - 1:
                time.sleep(1)  # Wait a bit before retrying
            else:
                return "error"

def modify_contig_taxa(row):
    prefix = prefixes.get(row['taxonomic_level'], '')
    return f"{prefix}{row['contig_taxa']}"

#EAF functions
def LongFormat(df, index, otu_col, value_col):
    return (df
     .set_index(index)
     .rename_axis([otu_col], axis=1).stack()
     .reset_index()
     .rename(columns={ 0: value_col}))

def reformate(df, index, otu_col, value_col):
    long = LongFormat(df, index, otu_col, value_col)
    long['treatment'] = long['samples'].apply(lambda x: "12C" if "12C" in x else "13C")
    long['day'] = long['samples'].apply(lambda x: "DAY7" if "DAY7" in x else "DAY0")
    long['fraction'] = long['samples'].str.extract(r'(\d+-\d+|\d+)$', expand=False)
    
    # Step 1: Create a mask indicating rows with count == 0
    zero_count_mask = long[value_col] == 0
    
    # Step 2: Identify 'samples' and 'organism' groups with any zero count
    groups_with_zero = long[zero_count_mask][['samples', 'organism']].drop_duplicates()
    
    # Step 3: Filter out these groups from the original DataFrame
    groups_with_zero['to_drop'] = True
    long = long.merge(groups_with_zero, on=['samples', 'organism'], how='left')
    
    long_filtered = long[long['to_drop'].isna()].drop(columns=['to_drop'])

    return long_filtered

def joindensitydf(df, densitydf, index, otu_col, value_col, common_column1,common_column2):
    reform = reformate(df, index, otu_col, value_col)
    reform2 = pd.merge(reform, densitydf, on=[common_column1,common_column2], how='left')

    rel_abun_qpcr_norm = reform2.groupby(['day', common_column1, 'organism']).filter(lambda x: len(x) > 2)

    rel_abun_qpcr_norm['count_qpcr'] = rel_abun_qpcr_norm['count'] * rel_abun_qpcr_norm['qpcr_ratio']

    # Group by 'day', 'treatment', 'organism' and calculate the sum for normalization
    grouped_sum = rel_abun_qpcr_norm.groupby(['day', common_column1, 'organism'])['count_qpcr'].transform('sum')

    # Calculate the normalized count
    rel_abun_qpcr_norm['count_qpcr_norm'] = rel_abun_qpcr_norm['count_qpcr'] / grouped_sum
    return rel_abun_qpcr_norm[['organism',common_column1,'density','count_qpcr_norm']]

def org_dna_denisity(df, densitydf, index, otu_col, value_col, common_column1,common_column2):
    joined_df=joindensitydf(df, densitydf, index, otu_col, value_col, common_column1,common_column2)
    # Group by contig and treatment, and calculate the sum of the product of rel_abun_qpcr_norm and density
    joined_df['weighted'] = joined_df['count_qpcr_norm'] * joined_df['density']
    grouped = joined_df.groupby(['organism', 'treatment']).apply(lambda x: (x['weighted']).sum())
    grouped = grouped.reset_index().rename(columns={ 0: 'weighted'})
    pivot_wider_df = grouped.pivot(index='organism', columns='treatment', values='weighted').reset_index()
    return pivot_wider_df

def AtomFractionExcess(df):
    #Shift
    df['shift'] = df['13C'] - df['12C']
    #Natural abundance molecular weight of each taxon: unlabeled
    df['M_light'] = 0.496 * df['GC'] + 307.691
    #Theoretical maximum molecular weight of fully labeled DNA
    df['M_HeavyMax'] = -0.4987282*df['GC'] + 9.974564 + df['M_light']
    #Molecular weight of DNA of taxon in labeled treatment
    df['M_Lab'] = (df['shift']/df['12C'] + 1) * df['M_light']
    df['EAF'] = (df['M_Lab'] - df['M_light']) / (df['M_HeavyMax'] - df['M_light']) * (1 - 0.01111233)
    return df

# Use coverage file from anvio to create a relative abundance dataframe. Save as a csv

In [5]:
# Define old and new names
oldnames = [
    "CLEAN_DAY7_DO_0_12C_CELL_ENRICHED_1_6", 
    "CLEAN_DAY7_DO_0_12C_CELL_ENRICHED_7",
    "CLEAN_DAY7_DO_0_12C_CELL_ENRICHED_8", 
    "CLEAN_DAY7_DO_0_12C_CELL_ENRICHED_9",
    "CLEAN_DAY7_DO_0_12C_CELL_ENRICHED_10_12",
    "CLEAN_DAY7_DO_0_13C_CELL_ENRICHED_1_5", 
    "CLEAN_DAY7_DO_0_13C_CELL_ENRICHED_6",
    "CLEAN_DAY7_DO_0_13C_CELL_ENRICHED_7", 
    "CLEAN_DAY7_DO_0_13C_CELL_ENRICHED_8",
    "CLEAN_DAY7_DO_0_13C_CELL_ENRICHED_9_12"
]

newnames = [
    "DAY7_DO_0_12C_CELL_ENRICHED_6", 
    "DAY7_DO_0_12C_CELL_ENRICHED_7",
    "DAY7_DO_0_12C_CELL_ENRICHED_8", 
    "DAY7_DO_0_12C_CELL_ENRICHED_9",
    "DAY7_DO_0_12C_CELL_ENRICHED_10",
    "DAY7_DO_0_13C_CELL_ENRICHED_5", 
    "DAY7_DO_0_13C_CELL_ENRICHED_6",
    "DAY7_DO_0_13C_CELL_ENRICHED_7", 
    "DAY7_DO_0_13C_CELL_ENRICHED_8",
    "DAY7_DO_0_13C_CELL_ENRICHED_9"
]

In [6]:
# Read the data
contig_cov = pd.read_csv("/projects/luo_lab/Siders_data/data/processed/anvio/merged_profile_db/drep/drep_contigs_removed-COVs.txt", sep="\t")

# Rename columns
contig_cov.rename(columns=dict(zip(oldnames, newnames)), inplace=True)

# Convert to DataFrame with contig as row index
contig_cov_transpose = contig_cov.set_index("contig").transpose().reset_index()

# Create a relative abundance table of all contigs in cell enrichment only

In [7]:
# Normalize the data
normalized_data = normalize(contig_cov_transpose.drop(columns="index"), axis=1, norm='l1')
cell_relative_abundance = pd.DataFrame(normalized_data, columns=contig_cov_transpose.columns[1:])
cell_relative_abundance["samples"] = contig_cov.columns[1:]

# Reorder columns to have samples first
cell_relative_abundance = cell_relative_abundance[["samples"] + list(cell_relative_abundance.columns[:-1])]

In [8]:
cell_relative_abundance_filter=cell_relative_abundance[cell_relative_abundance['samples'] != 'CLEAN_DAY0_DO_0_ENV_CELL_CONTROL_NONE']
cell_relative_abundance_filter

# Calculate the sum of each column
column_sums = cell_relative_abundance_filter.sum()

# Identify columns with a sum of zero
columns_to_drop = column_sums[column_sums == 0].index

# Drop the columns
cell_relative_abundance_filter = cell_relative_abundance_filter.drop(columns=columns_to_drop)

# Save to CSV
cell_relative_abundance_filter.to_csv("/projects/luo_lab/Siders_data/results/tables/drep_contigs_removed_rel_abun.csv", index=False)
cell_relative_abundance_filter

contig,samples,day0_DO_0_env_cell_control_000000000047,day0_DO_0_env_cell_control_000000000072,day0_DO_0_env_cell_control_000000000074,day0_DO_0_env_cell_control_000000000077,day0_DO_0_env_cell_control_000000000078,day0_DO_0_env_cell_control_000000000080,day0_DO_0_env_cell_control_000000000082,day0_DO_0_env_cell_control_000000000083,day0_DO_0_env_cell_control_000000000084,...,day7_DO_0_13C_cell_enriched_000001368152,day7_DO_0_13C_cell_enriched_000001368168,day7_DO_0_13C_cell_enriched_000001368172,day7_DO_0_13C_cell_enriched_000001368179,day7_DO_0_13C_cell_enriched_000001368190,day7_DO_0_13C_cell_enriched_000001368193,day7_DO_0_13C_cell_enriched_000001368197,day7_DO_0_13C_cell_enriched_000001368198,day7_DO_0_13C_cell_enriched_000001368200,day7_DO_0_13C_cell_enriched_000001368215
1,DAY7_DO_0_12C_CELL_ENRICHED_10,0.00000e+00,0.00000e+00,0.00000e+00,4.87405e-05,0.00000e+00,0.00000e+00,0.00000e+00,0.00000e+00,0.00000e+00,...,0.00000e+00,0.00000e+00,0.00000e+00,1.05815e-07,5.38950e-07,2.49808e-06,5.82673e-08,0.00000e+00,9.12268e-06,0.00000e+00
2,DAY7_DO_0_12C_CELL_ENRICHED_6,0.00000e+00,0.00000e+00,0.00000e+00,0.00000e+00,0.00000e+00,6.10148e-08,0.00000e+00,0.00000e+00,0.00000e+00,...,0.00000e+00,0.00000e+00,0.00000e+00,0.00000e+00,0.00000e+00,0.00000e+00,3.20580e-05,0.00000e+00,0.00000e+00,0.00000e+00
3,DAY7_DO_0_12C_CELL_ENRICHED_7,0.00000e+00,0.00000e+00,0.00000e+00,0.00000e+00,0.00000e+00,0.00000e+00,0.00000e+00,0.00000e+00,0.00000e+00,...,0.00000e+00,0.00000e+00,0.00000e+00,0.00000e+00,0.00000e+00,0.00000e+00,0.00000e+00,0.00000e+00,0.00000e+00,0.00000e+00
4,DAY7_DO_0_12C_CELL_ENRICHED_8,1.05294e-06,9.40815e-07,0.00000e+00,0.00000e+00,0.00000e+00,0.00000e+00,0.00000e+00,0.00000e+00,0.00000e+00,...,0.00000e+00,5.77501e-07,0.00000e+00,2.54617e-06,0.00000e+00,1.45697e-07,0.00000e+00,1.80899e-06,0.00000e+00,4.01915e-07
5,DAY7_DO_0_12C_CELL_ENRICHED_9,0.00000e+00,6.53058e-08,0.00000e+00,2.52107e-06,0.00000e+00,0.00000e+00,0.00000e+00,0.00000e+00,0.00000e+00,...,0.00000e+00,0.00000e+00,0.00000e+00,0.00000e+00,2.48689e-06,3.73381e-06,0.00000e+00,0.00000e+00,3.39019e-07,0.00000e+00
6,DAY7_DO_0_13C_CELL_ENRICHED_5,0.00000e+00,0.00000e+00,0.00000e+00,0.00000e+00,0.00000e+00,0.00000e+00,0.00000e+00,0.00000e+00,0.00000e+00,...,0.00000e+00,0.00000e+00,6.29825e-06,0.00000e+00,3.97965e-07,0.00000e+00,9.33642e-06,0.00000e+00,0.00000e+00,0.00000e+00
7,DAY7_DO_0_13C_CELL_ENRICHED_6,0.00000e+00,0.00000e+00,1.96868e-06,0.00000e+00,0.00000e+00,3.71164e-07,8.68527e-07,2.02310e-07,2.32371e-07,...,6.32649e-08,2.13190e-06,1.36973e-06,0.00000e+00,0.00000e+00,0.00000e+00,3.57620e-06,0.00000e+00,0.00000e+00,0.00000e+00
8,DAY7_DO_0_13C_CELL_ENRICHED_7,2.53416e-07,0.00000e+00,0.00000e+00,0.00000e+00,1.38546e-07,4.23462e-07,0.00000e+00,0.00000e+00,0.00000e+00,...,8.21407e-07,5.30166e-07,2.40594e-07,1.96130e-06,1.50183e-07,0.00000e+00,2.83001e-08,2.39617e-06,0.00000e+00,1.15052e-06
9,DAY7_DO_0_13C_CELL_ENRICHED_8,8.68853e-07,2.87016e-07,0.00000e+00,5.08690e-08,0.00000e+00,0.00000e+00,0.00000e+00,0.00000e+00,0.00000e+00,...,0.00000e+00,0.00000e+00,0.00000e+00,2.62293e-08,3.57247e-06,3.52167e-06,0.00000e+00,6.25654e-07,0.00000e+00,1.74583e-06
10,DAY7_DO_0_13C_CELL_ENRICHED_9,0.00000e+00,0.00000e+00,0.00000e+00,4.37940e-05,0.00000e+00,0.00000e+00,0.00000e+00,0.00000e+00,0.00000e+00,...,0.00000e+00,0.00000e+00,0.00000e+00,0.00000e+00,0.00000e+00,4.36630e-06,0.00000e+00,0.00000e+00,9.00764e-06,0.00000e+00


# Create density data frame

In [9]:
# Create the cell_density_df DataFrame
cell_density_df = pd.DataFrame({
    "fraction": ["6", "7", "8", "9", "10", "5", "6", "7", "8", "9"],
    "treatment": ["12C", "12C", "12C", "12C", "12C", "13C", "13C", "13C", "13C", "13C"],
    "density": [1.71328932, 1.70782552, 1.70236172, 1.69580516, 1.69034136,
                1.71547484, 1.70891828, 1.70345448, 1.69689792, 1.69143412],
    "qpcr_ratio": [0.038416781, 0.186560093, 0.987937393, 1, 0.113818084,
                   0.103393172, 1, 0.368097202, 0.224894639, 0.321954999],
    "filtrate_type": ["cell fraction"] * 10
})

# Remove rows where fraction is 12 and drop filtrate_type column
cell_density_df = cell_density_df[cell_density_df["fraction"] != "12"].drop(columns=["filtrate_type"])

# Save to CSV
cell_density_df.to_csv("/projects/luo_lab/Siders_data/results/tables/cell_density_table2.csv", index=False)


# Create a relative abundance table of the MAGs only

Create a data frame that has the lowest taxanomic level for each MAG

In [31]:
#!python3 /projects/luo_lab/Rogers_SidersViralAnalysis_XXXX_20XX/data/processed/Taxonomy/metadata/gtdb_to_ncbi_majority_vote.py --gtdbtk_output_dir /projects/luo_lab/Rogers_SidersViralAnalysis_XXXX_20XX/data/processed/Taxonomy/MAG/drep --bac120_metadata_file /projects/luo_lab/Rogers_SidersViralAnalysis_XXXX_20XX/data/processed/Taxonomy/metadata/bac120_metadata_r214.tar.gz --ar53_metadata_file /projects/luo_lab/Rogers_SidersViralAnalysis_XXXX_20XX/data/processed/Taxonomy/metadata/ar53_metadata_r214.tar.gz --output_file /projects/luo_lab/Rogers_SidersViralAnalysis_XXXX_20XX/data/processed/Taxonomy/MAG/drep/gtdbtk_to_ncbi_taxonomy.csv --gtdbtk_prefix gtdbtk

In [81]:
#Call in gtdbtk to ncbi prediction for cell enrichment MAGs. To make this I used this command:python3 gtdb_to_ncbi_majority_vote.py --gtdbtk_output_dir ../MAG --bac120_metadata_file bac120_metadata_r214.tar.gz --ar53_metadata_file ar53_metadata_r214.tar.gz --output_file ../MAG/gtdbtk_to_ncbi_taxonomy.csv --gtdbtk_prefix gtdbtk
mag_taxa = pd.read_csv("/projects/luo_lab/Siders_data/data/processed/taxonomy/MAG/gtdbtk_to_ncbi_taxonomy.csv", sep = '\t')
mag_taxa_sub=mag_taxa[["Genome ID", "Majority vote NCBI classification", "GTDB classification"]]
# Split the 'Majority vote NCBI classification' column into multiple columns
ncbi_cols = ['ncbi_domain', 'ncbi_phylum', 'ncbi_class', 'ncbi_order', 'ncbi_family', 'ncbi_genus', 'ncbi_species']
gtdb_cols = ['gtdb_domain', 'gtdb_phylum', 'gtdb_class', 'gtdb_order', 'gtdb_family', 'gtdb_genus', 'gtdb_species']

mag_taxa_sub[ncbi_cols] = mag_taxa['Majority vote NCBI classification'].str.split(';', expand=True)
mag_taxa_sub[gtdb_cols] = mag_taxa['GTDB classification'].str.split(';', expand=True)

# # Rename the columns
mag_taxa_sub = mag_taxa_sub.rename(columns={'Genome ID': 'MAG'})

# Define the replacement function
def replace_and_capitalize(match):
    if match.group(2):
        return f"{match.group(1).upper()}_{match.group(2)}"
    else:
        return pd.NA

for col in ncbi_cols:
    mag_taxa_sub[col] = mag_taxa_sub[col].str.replace(r"(.)__(.*)", replace_and_capitalize, regex=True)

for col in gtdb_cols:
    mag_taxa_sub[col] = mag_taxa_sub[col].str.replace(r"(.)__(.*)", replace_and_capitalize, regex=True)

# Replace empty strings with NaN
mag_taxa_sub = mag_taxa_sub.replace("", pd.NA)
# # Create 'taxonomy' column by coalescing across the classification columns
mag_taxa_sub['MAG_ncbi_taxa'] = mag_taxa_sub[['ncbi_species', 'ncbi_genus', 'ncbi_family', 'ncbi_order', 'ncbi_class', 'ncbi_phylum', 'ncbi_domain']].bfill(axis=1).iloc[:, 0]
mag_taxa_sub['MAG_gtdb_taxa'] = mag_taxa_sub[['gtdb_species', 'gtdb_genus', 'gtdb_family', 'gtdb_order', 'gtdb_class', 'gtdb_phylum', 'gtdb_domain']].bfill(axis=1).iloc[:, 0]
mag_taxa_sub

/tmp/ipykernel_1279723/3938357507.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mag_taxa_sub[ncbi_cols] = mag_taxa['Majority vote NCBI classification'].str.split(';', expand=True)
/tmp/ipykernel_1279723/3938357507.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mag_taxa_sub[ncbi_cols] = mag_taxa['Majority vote NCBI classification'].str.split(';', expand=True)
/tmp/ipykernel_1279723/3938357507.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try usin

,MAG,Majority vote NCBI classification,GTDB classification,ncbi_domain,ncbi_phylum,ncbi_class,ncbi_order,ncbi_family,ncbi_genus,ncbi_species,gtdb_domain,gtdb_phylum,gtdb_class,gtdb_order,gtdb_family,gtdb_genus,gtdb_species,MAG_ncbi_taxa,MAG_gtdb_taxa
0,day7-DO-0-12C-cell-enriched_bin.13,d__Archaea;p__;c__;o__;f__;g__;s__,d__Archaea;p__Nanoarchaeota;c__Nanoarchaeia;o_...,D_Archaea,NaN,NaN,NaN,NaN,NaN,NaN,D_Archaea,P_Nanoarchaeota,C_Nanoarchaeia,O_Pacearchaeales,F_GW2011-AR1,NaN,NaN,D_Archaea,F_GW2011-AR1
1,day7-DO-0-12C-cell-enriched_bin.43,d__Archaea;p__Candidatus Woesearchaeota;c__;o_...,d__Archaea;p__Nanoarchaeota;c__Nanoarchaeia;o_...,D_Archaea,P_Candidatus Woesearchaeota,NaN,NaN,NaN,NaN,NaN,D_Archaea,P_Nanoarchaeota,C_Nanoarchaeia,O_SCGC-AAA011-G17,F_GW2011-AR18,NaN,NaN,P_Candidatus Woesearchaeota,F_GW2011-AR18
2,day7-DO-0-12C-cell-enriched_bin.45,d__Archaea;p__;c__;o__;f__;g__;s__,d__Archaea;p__Nanoarchaeota;c__Nanoarchaeia;o_...,D_Archaea,NaN,NaN,NaN,NaN,NaN,NaN,D_Archaea,P_Nanoarchaeota,C_Nanoarchaeia,O_Pacearchaeales,F_GW2011-AR1,G_JAGVXH01,NaN,D_Archaea,G_JAGVXH01
3,day7-DO-0-13C-cell-enriched_bin.66,d__Archaea;p__Candidatus Altiarchaeota;c__;o__...,d__Archaea;p__Altiarchaeota;c__Altiarchaeia;o_...,D_Archaea,P_Candidatus Altiarchaeota,NaN,NaN,NaN,NaN,NaN,D_Archaea,P_Altiarchaeota,C_Altiarchaeia,O_IMC4,F_SCGC-AAA252-I15,G_JAHIYG01,NaN,P_Candidatus Altiarchaeota,G_JAHIYG01
4,day0-DO-0-env-cell-control_bin.20,d__Bacteria;p__Proteobacteria;c__Gammaproteoba...,d__Bacteria;p__Pseudomonadota;c__Gammaproteoba...,D_Bacteria,P_Proteobacteria,C_Gammaproteobacteria,NaN,NaN,NaN,NaN,D_Bacteria,P_Pseudomonadota,C_Gammaproteobacteria,O_GCF-002020875,F_GCF-002020875,NaN,NaN,C_Gammaproteobacteria,F_GCF-002020875
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153,day7-DO-0-12C-cell-enriched_bin.51,d__Bacteria;p__Lentisphaerae;c__;o__;f__;g__;s__,d__Bacteria;p__Verrucomicrobiota;c__Kiritimati...,D_Bacteria,P_Lentisphaerae,NaN,NaN,NaN,NaN,NaN,D_Bacteria,P_Verrucomicrobiota,C_Kiritimatiellia,O_UBA8416,F_JAIOPI01,NaN,NaN,P_Lentisphaerae,F_JAIOPI01
154,day7-DO-0-13C-cell-enriched_bin.2,d__Bacteria;p__Verrucomicrobia;c__;o__;f__;g__...,d__Bacteria;p__Verrucomicrobiota;c__Kiritimati...,D_Bacteria,P_Verrucomicrobia,NaN,NaN,NaN,NaN,NaN,D_Bacteria,P_Verrucomicrobiota,C_Kiritimatiellia,O_UBA8416,F_UBA8416,G_JABGOW01,NaN,P_Verrucomicrobia,G_JABGOW01
155,day7-DO-0-13C-cell-enriched_bin.26,d__Bacteria;p__Planctomycetes;c__Phycisphaerae...,d__Bacteria;p__Planctomycetota;c__Phycisphaera...,D_Bacteria,P_Planctomycetes,C_Phycisphaerae,NaN,NaN,NaN,NaN,D_Bacteria,P_Planctomycetota,C_Phycisphaerae,O_Sedimentisphaerales,F_SG8-4,G_JAIPFM01,NaN,C_Phycisphaerae,G_JAIPFM01
156,day7-DO-0-13C-cell-enriched_bin.4,d__Bacteria;p__Planctomycetes;c__Planctomyceti...,d__Bacteria;p__Planctomycetota;c__Planctomycet...,D_Bacteria,P_Planctomycetes,C_Planctomycetia,O_Pirellulales,NaN,NaN,NaN,D_Bacteria,P_Planctomycetota,C_Planctomycetia,O_Pirellulales,F_UBA1268,G_M30B53,NaN,O_Pirellulales,G_M30B53


In [82]:
# # Select the desired columns and rename them
mag_taxa_sub = mag_taxa_sub[['MAG', 'MAG_ncbi_taxa', 'MAG_gtdb_taxa','ncbi_domain', 'ncbi_phylum', 'ncbi_class', 'ncbi_order', 'ncbi_family', 'ncbi_genus', 'ncbi_species']]
taxa=['domain', 'phylum', 'class', 'order', 'family', 'genus', 'species']

mag_taxa_sub.rename(columns=dict(zip(ncbi_cols, taxa_cols)), inplace=True)

# Define the new column 'MAG_taxa' using a lambda function
mag_taxa_sub['MAG_taxa_combined'] = mag_taxa_sub.apply(
    lambda row: row['MAG_gtdb_taxa'] if row['MAG_ncbi_taxa'] == row['MAG_gtdb_taxa'] 
    else f"{row['MAG_gtdb_taxa']} (NCBI: {row['MAG_ncbi_taxa']})", axis=1
)

# Create mask for rows starting with 'Candidatus '
mask = mag_taxa_sub['MAG_ncbi_taxa'].str.lower().str.startswith('candidatus ')

# For rows starting with 'Candidatus ', keep the first two words
mag_taxa_sub.loc[mask, 'MAG_ncbi_taxa'] = mag_taxa_sub.loc[mask, 'MAG_ncbi_taxa'].str.split(' ').str[:2].str.join(' ')

# For remaining rows, keep only the first word
mag_taxa_sub.loc[~mask, 'MAG_ncbi_taxa'] = mag_taxa_sub.loc[~mask, 'MAG_ncbi_taxa'].str.split(' ').str[0]

mag_taxa_sub['MAG'] = mag_taxa_sub['MAG'].str.replace('.', '_')
mag_taxa_sub=mag_taxa_sub[['MAG','domain', 'phylum', 'class', 'order', 'family', 'genus', 'species','MAG_taxa_combined']]
mag_taxa_sub

/tmp/ipykernel_1279723/1877451193.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mag_taxa_sub.rename(columns=dict(zip(ncbi_cols, taxa_cols)), inplace=True)
/tmp/ipykernel_1279723/1877451193.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mag_taxa_sub['MAG_taxa_combined'] = mag_taxa_sub.apply(
/tmp/ipykernel_1279723/1877451193.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/

,MAG,domain,phylum,class,order,family,genus,species,MAG_taxa_combined
0,day7-DO-0-12C-cell-enriched_bin_13,D_Archaea,NaN,NaN,NaN,NaN,NaN,NaN,F_GW2011-AR1 (NCBI: D_Archaea)
1,day7-DO-0-12C-cell-enriched_bin_43,D_Archaea,P_Candidatus Woesearchaeota,NaN,NaN,NaN,NaN,NaN,F_GW2011-AR18 (NCBI: P_Candidatus Woesearchaeota)
2,day7-DO-0-12C-cell-enriched_bin_45,D_Archaea,NaN,NaN,NaN,NaN,NaN,NaN,G_JAGVXH01 (NCBI: D_Archaea)
3,day7-DO-0-13C-cell-enriched_bin_66,D_Archaea,P_Candidatus Altiarchaeota,NaN,NaN,NaN,NaN,NaN,G_JAHIYG01 (NCBI: P_Candidatus Altiarchaeota)
4,day0-DO-0-env-cell-control_bin_20,D_Bacteria,P_Proteobacteria,C_Gammaproteobacteria,NaN,NaN,NaN,NaN,F_GCF-002020875 (NCBI: C_Gammaproteobacteria)
...,...,...,...,...,...,...,...,...,...
153,day7-DO-0-12C-cell-enriched_bin_51,D_Bacteria,P_Lentisphaerae,NaN,NaN,NaN,NaN,NaN,F_JAIOPI01 (NCBI: P_Lentisphaerae)
154,day7-DO-0-13C-cell-enriched_bin_2,D_Bacteria,P_Verrucomicrobia,NaN,NaN,NaN,NaN,NaN,G_JABGOW01 (NCBI: P_Verrucomicrobia)
155,day7-DO-0-13C-cell-enriched_bin_26,D_Bacteria,P_Planctomycetes,C_Phycisphaerae,NaN,NaN,NaN,NaN,G_JAIPFM01 (NCBI: C_Phycisphaerae)
156,day7-DO-0-13C-cell-enriched_bin_4,D_Bacteria,P_Planctomycetes,C_Planctomycetia,O_Pirellulales,NaN,NaN,NaN,G_M30B53 (NCBI: O_Pirellulales)


Create the normalized abundance data frame for shift plot

In [14]:
mag_contigs = pd.read_csv("/projects/luo_lab/Siders_data/data/processed/MAGs/drep/extra_files/binning_results.txt", 
                          header=None,sep='\t', names=['organism', 'MAG'])
# Select the columns needed for the calcuations:
taxa=mag_taxa_sub[['MAG','MAG_taxa_combined']]
# Transform the data
MAG_relative_abundance = (
    cell_relative_abundance_filter
    .melt(id_vars="samples", var_name="organism", value_name="relative_abun")
    .merge(mag_contigs, on="organism")
    .drop(columns=["organism"])
    .groupby(["samples", "MAG"], as_index=False)["relative_abun"]
    .sum()
)

# Add treatment and day columns
MAG_relative_abundance["treatment"] = MAG_relative_abundance["samples"].apply(lambda x: "12C" if "12C" in x else "13C")
MAG_relative_abundance["day"] = MAG_relative_abundance["samples"].apply(lambda x: "DAY7" if "DAY7" in x else "DAY0")

# # Extract sample and fraction
MAG_relative_abundance[["sample", "fraction"]] = MAG_relative_abundance["samples"].str.extract(r"(.*)_([^_]+)$")

#Merge with taxa and density_values
MAG_relative_abundance = (
     MAG_relative_abundance
     .merge(taxa, how="left")
     .merge(cell_density_df, how="left")
 )

# Normalize relative abundance with qPCR ratio
qpcr_sum = (MAG_relative_abundance["relative_abun"] * MAG_relative_abundance["qpcr_ratio"]).sum()
MAG_relative_abundance["rel_abun_qpcr_norm"] = MAG_relative_abundance["relative_abun"] * MAG_relative_abundance["qpcr_ratio"] / qpcr_sum
MAG_relative_abundance = MAG_relative_abundance[MAG_relative_abundance['MAG_taxa_combined'].notna() & (MAG_relative_abundance['MAG_taxa_combined'] != '')]
MAG_relative_abundance.to_csv("/projects/luo_lab/Siders_data/results/tables/drep_contigs_removed_MAG_rel_abun_to_contigs.csv", index=False)
MAG_relative_abundance

,samples,MAG,relative_abun,treatment,day,sample,fraction,MAG_taxa_combined,density,qpcr_ratio,rel_abun_qpcr_norm
0,DAY7_DO_0_12C_CELL_ENRICHED_10,day0-DO-0-env-cell-control_bin_1,1.12134e-06,12C,DAY7,DAY7_DO_0_12C_CELL_ENRICHED,10,G_QWPN01 (NCBI: G_Candidatus Kuenenia),1.69034,0.11382,1.22320e-07
1,DAY7_DO_0_12C_CELL_ENRICHED_10,day0-DO-0-env-cell-control_bin_100,1.36469e-07,12C,DAY7,DAY7_DO_0_12C_CELL_ENRICHED,10,O_JAIPIQ01 (NCBI: P_Candidatus Marinimicrobia),1.69034,0.11382,1.48865e-08
2,DAY7_DO_0_12C_CELL_ENRICHED_10,day0-DO-0-env-cell-control_bin_101,6.63199e-06,12C,DAY7,DAY7_DO_0_12C_CELL_ENRICHED,10,G_Synechococcus_C (NCBI: G_Synechococcus),1.69034,0.11382,7.23438e-07
3,DAY7_DO_0_12C_CELL_ENRICHED_10,day0-DO-0-env-cell-control_bin_102,1.61140e-07,12C,DAY7,DAY7_DO_0_12C_CELL_ENRICHED,10,C_Desulfarculia (NCBI: C_Deltaproteobacteria),1.69034,0.11382,1.75776e-08
4,DAY7_DO_0_12C_CELL_ENRICHED_10,day0-DO-0-env-cell-control_bin_103,7.91856e-07,12C,DAY7,DAY7_DO_0_12C_CELL_ENRICHED,10,O_Kiritimatiellales,1.69034,0.11382,8.63781e-08
...,...,...,...,...,...,...,...,...,...,...,...
1575,DAY7_DO_0_13C_CELL_ENRICHED_9,day7-DO-0-13C-cell-enriched_bin_57,6.43244e-03,13C,DAY7,DAY7_DO_0_13C_CELL_ENRICHED,9,G_4572-104 (NCBI: P_Tenericutes),1.69143,0.32195,1.98480e-03
1576,DAY7_DO_0_13C_CELL_ENRICHED_9,day7-DO-0-13C-cell-enriched_bin_6,6.19092e-06,13C,DAY7,DAY7_DO_0_13C_CELL_ENRICHED,9,G_UBA6154 (NCBI: O_Candidatus Nanopelagicales),1.69143,0.32195,1.91028e-06
1577,DAY7_DO_0_13C_CELL_ENRICHED_9,day7-DO-0-13C-cell-enriched_bin_61,2.82757e-05,13C,DAY7,DAY7_DO_0_13C_CELL_ENRICHED,9,O_LZORAL124-64-63 (NCBI: D_Bacteria),1.69143,0.32195,8.72479e-06
1578,DAY7_DO_0_13C_CELL_ENRICHED_9,day7-DO-0-13C-cell-enriched_bin_66,3.63616e-05,13C,DAY7,DAY7_DO_0_13C_CELL_ENRICHED,9,G_JAHIYG01 (NCBI: P_Candidatus Altiarchaeota),1.69143,0.32195,1.12198e-05


# Add taxonomy to contig relative abundance

In [69]:
col_names=['classified','organism','taxid','score','identifiers','Assessions','matching_fragment_sequence','contig_taxa']
cell_kaiju_taxa=pd.read_csv("/projects/luo_lab/Siders_data/data/processed/taxonomy/contig/kaiju/kaiju_names.out", sep='\t', names=col_names) 
cell_kaiju_taxa2=cell_kaiju_taxa[['organism','contig_taxa']]
cell_kaiju_taxa2=cell_kaiju_taxa2[cell_kaiju_taxa2['contig_taxa'].notna()]
cell_kaiju_taxa2

,organism,contig_taxa
6,day0_DO_0_env_cell_control_000000000080,Bacteria; Actinomycetota; Actinomycetes; Micro...
10,day0_DO_0_env_cell_control_000000000086,Bacteria; Verrucomicrobiota; Verrucomicrobiia;...
11,day0_DO_0_env_cell_control_000000000090,Bacteria; Bacillota; Clostridia; Halanaerobial...
13,day0_DO_0_env_cell_control_000000000092,Bacteria; Pseudomonadota; Gammaproteobacteria;...
14,day0_DO_0_env_cell_control_000000000094,Bacteria; Chloroflexota; Chloroflexia; Chlorof...
...,...,...
606420,day7_DO_0_13C_cell_enriched_000001368138,Bacteria; Kiritimatiellota; Kiritimatiellia; K...
606423,day7_DO_0_13C_cell_enriched_000001368172,NA; NA; NA; NA; NA; NA; NA;
606425,day7_DO_0_13C_cell_enriched_000001368190,Bacteria; Thermodesulfobacteriota; Desulfovibr...
606426,day7_DO_0_13C_cell_enriched_000001368193,Bacteria; Bacteroidota; Sphingobacteriia; Sphi...


In [83]:
#Call in gtdbtk to ncbi prediction for cell enrichment MAGs. To make this I used this command:python3 gtdb_to_ncbi_majority_vote.py --gtdbtk_output_dir ../MAG --bac120_metadata_file bac120_metadata_r214.tar.gz --ar53_metadata_file ar53_metadata_r214.tar.gz --output_file ../MAG/gtdbtk_to_ncbi_taxonomy.csv --gtdbtk_prefix gtdbtk
# Split the 'Majority vote NCBI classification' column into multiple columns

taxa_cols=['domain', 'phylum', 'class', 'order', 'family', 'genus', 'species','blank']
cell_kaiju_taxa3=cell_kaiju_taxa2
#cell_kaiju_taxa[taxa_cols] =
split_cols=cell_kaiju_taxa3['contig_taxa'].str.split(';', expand=True, n=7)
cell_kaiju_taxa3[taxa_cols] = split_cols.reindex(columns=range(8))
# Remove leading whitespace (lstrip) from specific columns
cell_kaiju_taxa3[taxa_cols] = cell_kaiju_taxa3[taxa_cols].apply(lambda x: x.str.lstrip())

cell_kaiju_taxa3.replace('NA', np.nan, inplace=True)
cell_kaiju_taxa3=cell_kaiju_taxa3[cell_kaiju_taxa3['domain'].notna()]
cell_kaiju_taxa3=cell_kaiju_taxa3[['organism', 'contig_taxa', 'domain', 'phylum', 'class', 'order', 'family', 'genus', 'species']]

def add_prefix(df, cols):
    for col in cols:
        first_letter = col[0].upper()  # Get the first letter of the column name and capitalize it
        df[col] = df[col].apply(lambda x: f"{first_letter}_{x}" if pd.notna(x) and x != 'NaN' else x)
    return df

cell_kaiju_taxa3 = add_prefix(cell_kaiju_taxa3, ['domain', 'phylum', 'class', 'order', 'family', 'genus', 'species'])
cell_kaiju_taxa3



# Create 'taxonomy' column by coalescing across the classification columns
cell_kaiju_taxa3['contig_taxa'] = cell_kaiju_taxa3[[ 'genus', 'family', 'order', 'class', 'phylum', 'domain']].bfill(axis=1).iloc[:, 0]
cell_kaiju_taxa3.to_csv('/projects/luo_lab/Siders_data/results/tables/kaiju_contig_taxa.csv', index=False) 
cell_kaiju_taxa3



,organism,contig_taxa,domain,phylum,class,order,family,genus,species
6,day0_DO_0_env_cell_control_000000000080,G_Protaetiibacter,D_Bacteria,P_Actinomycetota,C_Actinomycetes,O_Micrococcales,F_Microbacteriaceae,G_Protaetiibacter,S_Protaetiibacter sp. SSC-01
10,day0_DO_0_env_cell_control_000000000086,G_Haloferula,D_Bacteria,P_Verrucomicrobiota,C_Verrucomicrobiia,O_Verrucomicrobiales,F_Verrucomicrobiaceae,G_Haloferula,S_Haloferula helveola
11,day0_DO_0_env_cell_control_000000000090,G_Halobacteroides,D_Bacteria,P_Bacillota,C_Clostridia,O_Halanaerobiales,F_Halobacteroidaceae,G_Halobacteroides,S_Halobacteroides halobius
13,day0_DO_0_env_cell_control_000000000092,G_Marinobacterium,D_Bacteria,P_Pseudomonadota,C_Gammaproteobacteria,O_Oceanospirillales,F_Oceanospirillaceae,G_Marinobacterium,S_Marinobacterium aestuarii
14,day0_DO_0_env_cell_control_000000000094,G_Roseiflexus,D_Bacteria,P_Chloroflexota,C_Chloroflexia,O_Chloroflexales,F_Roseiflexaceae,G_Roseiflexus,NaN
...,...,...,...,...,...,...,...,...,...
606415,day7_DO_0_13C_cell_enriched_000001368078,G_Desulfoferula,D_Bacteria,P_Thermodesulfobacteriota,C_Desulfarculia,O_Desulfarculales,F_Desulfarculaceae,G_Desulfoferula,S_Desulfoferula mesophila
606420,day7_DO_0_13C_cell_enriched_000001368138,G_Kiritimatiella,D_Bacteria,P_Kiritimatiellota,C_Kiritimatiellia,O_Kiritimatiellales,F_Kiritimatiellaceae,G_Kiritimatiella,S_Kiritimatiella glycovorans
606425,day7_DO_0_13C_cell_enriched_000001368190,G_Pseudodesulfovibrio,D_Bacteria,P_Thermodesulfobacteriota,C_Desulfovibrionia,O_Desulfovibrionales,F_Desulfovibrionaceae,G_Pseudodesulfovibrio,S_Pseudodesulfovibrio aespoeensis
606426,day7_DO_0_13C_cell_enriched_000001368193,G_Mucilaginibacter,D_Bacteria,P_Bacteroidota,C_Sphingobacteriia,O_Sphingobacteriales,F_Sphingobacteriaceae,G_Mucilaginibacter,S_Mucilaginibacter xinganensis


# Part 1: Calculate the EAF of contigs in the cell enriched fraction 

Step 1: Call in required data frames for EAF calculation of the cell enriched fraction


In [84]:
# Call in data frame of the gc content producted by seqkit: seqkit fx2tab --name --only-id --gc combined_cell_contigs_clean_headers.fa > combined_cell_contigs_gc_results.txt
GC_content=pd.read_csv("/projects/luo_lab/Siders_data/results/tables/drep_contigs_gc.txt", sep='\t',header=None, names=['organism','GC'])
GC_content['GC']=GC_content['GC']/100
GC_content

,organism,GC
0,day0_DO_0_env_cell_control_000000000017,0.5432
1,day0_DO_0_env_cell_control_000000000047,0.5106
2,day0_DO_0_env_cell_control_000000000072,0.5190
3,day0_DO_0_env_cell_control_000000000074,0.6227
4,day0_DO_0_env_cell_control_000000000077,0.2668
...,...,...
606426,day7_DO_0_13C_cell_enriched_000001368193,0.3937
606427,day7_DO_0_13C_cell_enriched_000001368197,0.6331
606428,day7_DO_0_13C_cell_enriched_000001368198,0.4803
606429,day7_DO_0_13C_cell_enriched_000001368200,0.2448


Step 2: Use the 'org_dna_density' function to calculate the mean DNA density of each contig

In [85]:
cell_relative_abundance_long=org_dna_denisity(cell_relative_abundance_filter, cell_density_df, 'samples','organism','count', 'treatment','fraction')
cell_relative_abundance_long

cell_relative_abundance_long = cell_relative_abundance_long[cell_relative_abundance_long['12C'].notna()]
cell_relative_abundance_long = cell_relative_abundance_long[cell_relative_abundance_long['13C'].notna()]

/tmp/ipykernel_1279723/1487431982.py:73: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped = joined_df.groupby(['organism', 'treatment']).apply(lambda x: (x['weighted']).sum())


In [87]:
cell_relative_abundance_long=pd.merge(cell_relative_abundance_long, GC_content)
cell_relative_abundance_long
test_sorted = cell_relative_abundance_long.sort_values(by='12C')
test_sorted

,organism,12C,13C,GC
31102,day7_DO_0_13C_cell_enriched_000000145017,1.69054,1.69588,0.4213
3558,day0_DO_0_env_cell_control_000000411487,1.69067,1.69183,0.3117
30345,day7_DO_0_13C_cell_enriched_000000091461,1.69088,1.70186,0.5081
9972,day0_DO_0_env_cell_control_000001473230,1.69091,1.69148,0.3530
11715,day0_DO_0_env_cell_control_000001898665,1.69098,1.69325,0.3171
...,...,...,...,...
14503,day7_DO_0_12C_cell_enriched_000000097819,1.71302,1.71016,0.6273
2947,day0_DO_0_env_cell_control_000000329791,1.71304,1.71045,0.6641
39008,day7_DO_0_13C_cell_enriched_000000814428,1.71309,1.70883,0.6202
15332,day7_DO_0_12C_cell_enriched_000000136616,1.71313,1.71300,0.6588


Step 3: Use the 'AtomFractionExcess' function to calculate the EAF of each contig and export to csv file to be used in R script for figure construction.

In [88]:
cell_atomic_fraction = pd.merge(mag_contigs,AtomFractionExcess(cell_relative_abundance_long), on=['organism'], how='outer')
#cell_atomic_fraction = cell_atomic_fraction[cell_atomic_fraction['12C'].notna()]
#cell_atomic_fraction = cell_atomic_fraction[cell_atomic_fraction['13C'].notna()]
# Replace empty strings in 'MAG' with "unbinned"
cell_atomic_fraction['MAG'] = cell_atomic_fraction['MAG'].fillna("unbinned")
cell_atomic_fraction
cell_atomic_fraction = cell_atomic_fraction[cell_atomic_fraction['EAF'].notna()]
cell_atomic_fraction.to_csv('/projects/luo_lab/Siders_data/results/tables/cell_eaf.csv', index=False) 
# mag_atomic_fraction.to_csv('../../R/output_files/mag_atomic_fraction_E.csv', index=False) 
test_sorted = cell_atomic_fraction.sort_values(by='12C')
test_sorted

,organism,MAG,12C,13C,GC,shift,M_light,M_HeavyMax,M_Lab,EAF
82871,day7_DO_0_13C_cell_enriched_000000145017,unbinned,1.69054,1.69588,0.4213,0.00533,307.89996,317.66441,308.87104,0.09835
12752,day0_DO_0_env_cell_control_000000411487,unbinned,1.69067,1.69183,0.3117,0.00117,307.84560,317.66471,308.05811,0.02140
81618,day7_DO_0_13C_cell_enriched_000000091461,unbinned,1.69088,1.70186,0.5081,0.01097,307.94302,317.66418,309.94147,0.20329
41415,day0_DO_0_env_cell_control_000001473230,unbinned,1.69091,1.69148,0.3530,0.00057,307.86609,317.66460,307.96992,0.01048
51472,day0_DO_0_env_cell_control_000001898665,unbinned,1.69098,1.69325,0.3171,0.00227,307.84828,317.66470,308.26150,0.04163
...,...,...,...,...,...,...,...,...,...,...
58503,day7_DO_0_12C_cell_enriched_000000097819,unbinned,1.71302,1.71016,0.6273,-0.00286,308.00214,317.66385,307.48795,-0.05263
10286,day0_DO_0_env_cell_control_000000329791,day0-DO-0-env-cell-control_bin_80,1.71304,1.71045,0.6641,-0.00258,308.02039,317.66375,307.55567,-0.04766
96361,day7_DO_0_13C_cell_enriched_000000814428,unbinned,1.71309,1.70883,0.6202,-0.00427,307.99862,317.66387,307.23140,-0.07850
59640,day7_DO_0_12C_cell_enriched_000000136616,unbinned,1.71313,1.71300,0.6588,-0.00013,308.01776,317.66377,307.99394,-0.00244


In [92]:
#Merg cell_enriched_eaf,  mag_taxa_sub, and cell_kraken2_taxa_sub
# Perform the full join (equivalent to a merge in pandas)
cell_atomic_fraction2 = pd.merge(cell_atomic_fraction, mag_taxa_sub[["MAG","MAG_taxa_combined"]], how='outer')
cell_atomic_fraction2 = pd.merge(cell_atomic_fraction2, cell_kaiju_taxa3, how='outer')
cell_atomic_fraction2


,organism,MAG,12C,13C,GC,shift,M_light,M_HeavyMax,M_Lab,EAF,MAG_taxa_combined,contig_taxa,domain,phylum,class,order,family,genus,species
0,day0_DO_0_env_cell_control_000000000080,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,G_Protaetiibacter,D_Bacteria,P_Actinomycetota,C_Actinomycetes,O_Micrococcales,F_Microbacteriaceae,G_Protaetiibacter,S_Protaetiibacter sp. SSC-01
1,day0_DO_0_env_cell_control_000000000086,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,G_Haloferula,D_Bacteria,P_Verrucomicrobiota,C_Verrucomicrobiia,O_Verrucomicrobiales,F_Verrucomicrobiaceae,G_Haloferula,S_Haloferula helveola
2,day0_DO_0_env_cell_control_000000000090,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,G_Halobacteroides,D_Bacteria,P_Bacillota,C_Clostridia,O_Halanaerobiales,F_Halobacteroidaceae,G_Halobacteroides,S_Halobacteroides halobius
3,day0_DO_0_env_cell_control_000000000092,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,G_Marinobacterium,D_Bacteria,P_Pseudomonadota,C_Gammaproteobacteria,O_Oceanospirillales,F_Oceanospirillaceae,G_Marinobacterium,S_Marinobacterium aestuarii
4,day0_DO_0_env_cell_control_000000000094,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,G_Roseiflexus,D_Bacteria,P_Chloroflexota,C_Chloroflexia,O_Chloroflexales,F_Roseiflexaceae,G_Roseiflexus,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
341972,NaN,day7-DO-0-13C-cell-enriched_bin_16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,G_CAJXII01 (NCBI: G_Arcobacter),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
341973,NaN,day7-DO-0-13C-cell-enriched_bin_18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,G_Xianfuyuplasma,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
341974,NaN,day7-DO-0-13C-cell-enriched_bin_28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,G_Sulfurimonas,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
341975,NaN,day7-DO-0-13C-cell-enriched_bin_40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,G_Halothiobacillus,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [94]:
# Apply the mutate logic using pandas' .apply and .loc
cell_atomic_fraction2['taxa'] = cell_atomic_fraction2.apply(
    lambda row: row['MAG_taxa_combined'] if pd.notna(row['MAG_taxa_combined']) else row['contig_taxa'], axis=1)
cell_atomic_fraction2['taxa'] = cell_atomic_fraction2['taxa'].replace("cellular organisms", pd.NA)
cell_atomic_fraction2['taxa'] = cell_atomic_fraction2['taxa'].apply(
    lambda x: x.replace("Unclassified ", "") if pd.notna(x) and "Unclassified" in x else x)
cell_atomic_fraction2['taxa'] = cell_atomic_fraction2['taxa'].apply(
    lambda x: x.replace("unclassified ", "") if pd.notna(x) and "unclassified" in x else x)
cell_atomic_fraction2['taxa'] = cell_atomic_fraction2['taxa'].str.replace(r'.*/\s*(.*$)', r'\1', regex=True)
cell_atomic_fraction2['MAG'] = cell_atomic_fraction2['MAG'].replace("", "unbinned")
# # Filter out rows where 'taxa' is NaN
cell_atomic_fraction2 = cell_atomic_fraction2.dropna(subset=['taxa'])
cell_atomic_fraction2 = cell_atomic_fraction2.dropna(subset=['organism'])
# #save the resulting DataFrame
cell_atomic_fraction2 = cell_atomic_fraction2[cell_atomic_fraction2['12C'].notna()]
cell_atomic_fraction2 = cell_atomic_fraction2[cell_atomic_fraction2['13C'].notna()]
cell_atomic_fraction2

,organism,MAG,12C,13C,GC,shift,M_light,M_HeavyMax,M_Lab,EAF,MAG_taxa_combined,contig_taxa,domain,phylum,class,order,family,genus,species,taxa
32,day0_DO_0_env_cell_control_000000000369,unbinned,1.69655,1.69530,0.4107,-0.00126,307.89471,317.66444,307.66636,-0.02311,NaN,G_Marinifilum,D_Bacteria,P_Bacteroidota,C_Bacteroidia,O_Marinilabiliales,F_Marinifilaceae,G_Marinifilum,S_Marinifilum fragile,G_Marinifilum
55,day0_DO_0_env_cell_control_000000000770,unbinned,1.70497,1.70732,0.5173,0.00235,307.94758,317.66415,308.37261,0.04326,NaN,G_Sediminispirochaeta,D_Bacteria,P_Spirochaetota,C_Spirochaetia,O_Spirochaetales,F_Spirochaetaceae,G_Sediminispirochaeta,S_Sediminispirochaeta smaragdinae,G_Sediminispirochaeta
56,day0_DO_0_env_cell_control_000000000775,day0-DO-0-env-cell-control_bin_57,1.70689,1.70816,0.5850,0.00127,307.98116,317.66397,308.21052,0.02342,F_Alkalispirochaetaceae (NCBI: F_Spirochaetaceae),G_Fibrobacter,D_Bacteria,P_Fibrobacterota,C_Fibrobacteria,O_Fibrobacterales,F_Fibrobacteraceae,G_Fibrobacter,S_Fibrobacter succinogenes,F_Alkalispirochaetaceae (NCBI: F_Spirochaetaceae)
68,day0_DO_0_env_cell_control_000000000851,day0-DO-0-env-cell-control_bin_22,1.70732,1.70851,0.6089,0.00119,307.99301,317.66390,308.20775,0.02196,G_JADHUC01 (NCBI: F_Pirellulaceae),G_Lignipirellula,D_Bacteria,P_Planctomycetota,C_Planctomycetia,O_Pirellulales,F_Pirellulaceae,G_Lignipirellula,S_Lignipirellula cremea,G_JADHUC01 (NCBI: F_Pirellulaceae)
69,day0_DO_0_env_cell_control_000000000876,unbinned,1.70083,1.70318,0.5205,0.00235,307.94917,317.66414,308.37444,0.04329,NaN,G_Shinella,D_Bacteria,P_Pseudomonadota,C_Alphaproteobacteria,O_Hyphomicrobiales,F_Rhizobiaceae,G_Shinella,S_Shinella zoogloeoides,G_Shinella
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
341805,day7_DO_0_13C_cell_enriched_000001366290,day7-DO-0-13C-cell-enriched_bin_34,1.69629,1.69635,0.3999,0.00006,307.88935,317.66447,307.90047,0.00113,G_SDB (NCBI: G_Candidatus Cloacimonas),G_Flavonifractor,D_Bacteria,P_Bacillota,C_Clostridia,O_Eubacteriales,F_Oscillospiraceae,G_Flavonifractor,S_Flavonifractor plautii,G_SDB (NCBI: G_Candidatus Cloacimonas)
341825,day7_DO_0_13C_cell_enriched_000001366614,unbinned,1.69663,1.69679,0.4843,0.00015,307.93121,317.66424,307.95929,0.00285,NaN,G_Draconibacterium,D_Bacteria,P_Bacteroidota,C_Bacteroidia,O_Marinilabiliales,F_Prolixibacteraceae,G_Draconibacterium,S_uncultured Draconibacterium sp.,G_Draconibacterium
341872,day7_DO_0_13C_cell_enriched_000001367621,day7-DO-0-13C-cell-enriched_bin_11,1.69484,1.69198,0.3396,-0.00287,307.85944,317.66464,307.33888,-0.05250,G_GCA-002708315 (NCBI: O_Bacteroidales),G_Candidatus Sulfidibacterium,D_Bacteria,P_Bacteroidota,NaN,O_Candidatus Sulfidibacteriales,NaN,G_Candidatus Sulfidibacterium,S_Candidatus Sulfidibacterium hydrothermale,G_GCA-002708315 (NCBI: O_Bacteroidales)
341895,day7_DO_0_13C_cell_enriched_000001368028,unbinned,1.69627,1.69402,0.3234,-0.00226,307.85141,317.66468,307.44176,-0.04128,NaN,G_Cloacibacterium,D_Bacteria,P_Bacteroidota,C_Flavobacteriia,O_Flavobacteriales,F_Weeksellaceae,G_Cloacibacterium,S_Cloacibacterium caeni,G_Cloacibacterium


In [95]:
cell_atomic_fraction3 = cell_atomic_fraction2.drop(columns=['MAG_taxa_combined', 'contig_taxa'])
cell_atomic_fraction3.to_csv('/projects/luo_lab/Siders_data/results/tables/cell_eaf_taxa.csv', index=False) 
cell_atomic_fraction3

,organism,MAG,12C,13C,GC,shift,M_light,M_HeavyMax,M_Lab,EAF,domain,phylum,class,order,family,genus,species,taxa
32,day0_DO_0_env_cell_control_000000000369,unbinned,1.69655,1.69530,0.4107,-0.00126,307.89471,317.66444,307.66636,-0.02311,D_Bacteria,P_Bacteroidota,C_Bacteroidia,O_Marinilabiliales,F_Marinifilaceae,G_Marinifilum,S_Marinifilum fragile,G_Marinifilum
55,day0_DO_0_env_cell_control_000000000770,unbinned,1.70497,1.70732,0.5173,0.00235,307.94758,317.66415,308.37261,0.04326,D_Bacteria,P_Spirochaetota,C_Spirochaetia,O_Spirochaetales,F_Spirochaetaceae,G_Sediminispirochaeta,S_Sediminispirochaeta smaragdinae,G_Sediminispirochaeta
56,day0_DO_0_env_cell_control_000000000775,day0-DO-0-env-cell-control_bin_57,1.70689,1.70816,0.5850,0.00127,307.98116,317.66397,308.21052,0.02342,D_Bacteria,P_Fibrobacterota,C_Fibrobacteria,O_Fibrobacterales,F_Fibrobacteraceae,G_Fibrobacter,S_Fibrobacter succinogenes,F_Alkalispirochaetaceae (NCBI: F_Spirochaetaceae)
68,day0_DO_0_env_cell_control_000000000851,day0-DO-0-env-cell-control_bin_22,1.70732,1.70851,0.6089,0.00119,307.99301,317.66390,308.20775,0.02196,D_Bacteria,P_Planctomycetota,C_Planctomycetia,O_Pirellulales,F_Pirellulaceae,G_Lignipirellula,S_Lignipirellula cremea,G_JADHUC01 (NCBI: F_Pirellulaceae)
69,day0_DO_0_env_cell_control_000000000876,unbinned,1.70083,1.70318,0.5205,0.00235,307.94917,317.66414,308.37444,0.04329,D_Bacteria,P_Pseudomonadota,C_Alphaproteobacteria,O_Hyphomicrobiales,F_Rhizobiaceae,G_Shinella,S_Shinella zoogloeoides,G_Shinella
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
341805,day7_DO_0_13C_cell_enriched_000001366290,day7-DO-0-13C-cell-enriched_bin_34,1.69629,1.69635,0.3999,0.00006,307.88935,317.66447,307.90047,0.00113,D_Bacteria,P_Bacillota,C_Clostridia,O_Eubacteriales,F_Oscillospiraceae,G_Flavonifractor,S_Flavonifractor plautii,G_SDB (NCBI: G_Candidatus Cloacimonas)
341825,day7_DO_0_13C_cell_enriched_000001366614,unbinned,1.69663,1.69679,0.4843,0.00015,307.93121,317.66424,307.95929,0.00285,D_Bacteria,P_Bacteroidota,C_Bacteroidia,O_Marinilabiliales,F_Prolixibacteraceae,G_Draconibacterium,S_uncultured Draconibacterium sp.,G_Draconibacterium
341872,day7_DO_0_13C_cell_enriched_000001367621,day7-DO-0-13C-cell-enriched_bin_11,1.69484,1.69198,0.3396,-0.00287,307.85944,317.66464,307.33888,-0.05250,D_Bacteria,P_Bacteroidota,NaN,O_Candidatus Sulfidibacteriales,NaN,G_Candidatus Sulfidibacterium,S_Candidatus Sulfidibacterium hydrothermale,G_GCA-002708315 (NCBI: O_Bacteroidales)
341895,day7_DO_0_13C_cell_enriched_000001368028,unbinned,1.69627,1.69402,0.3234,-0.00226,307.85141,317.66468,307.44176,-0.04128,D_Bacteria,P_Bacteroidota,C_Flavobacteriia,O_Flavobacteriales,F_Weeksellaceae,G_Cloacibacterium,S_Cloacibacterium caeni,G_Cloacibacterium


# Part 2: Calculate the EAF of the vOTUs present in the cell enriched fraction 

Step 1: Call in required data frames for EAF calculation of the cell enriched fraction

In [39]:
viral_ce_cov = pd.read_csv("/projects/luo_lab/Siders_data/data/processed/anvio/merged_profile_db/derep_viral_ce/derep_viral_ce-COVs.txt", sep="\t")

# Rename columns
viral_ce_cov.rename(columns=dict(zip(oldnames, newnames)), inplace=True)
viral_ce_cov.to_csv("/projects/luo_lab/Siders_data/results/tables/derep_viral_ce-COVs_clean_names2.csv", index=False)

# Rename columns
viral_ce_cov.rename(columns=dict(zip(oldnames, newnames)), inplace=True)
#viral_ce_cov = viral_ce_cov.drop(columns=columns_to_remove)
# Save to CSV
#viral_ce_cov.to_csv("../../R/input_files/vcell_retentate_cov_clean_names.csv", index=False)

# Convert to DataFrame with contig as row index
viral_ce_relative_abundance = viral_ce_cov.set_index("contig").transpose().reset_index()

# Normalize the data
normalized_data = normalize(viral_ce_relative_abundance.drop(columns="index"), axis=1, norm='l1')
viral_ce_relative_abundance = pd.DataFrame(normalized_data, columns=viral_ce_relative_abundance.columns[1:])
viral_ce_relative_abundance["samples"] = viral_ce_cov.columns[1:]

# Reorder columns to have samples first
viral_ce_relative_abundance = viral_ce_relative_abundance[["samples"] + list(viral_ce_relative_abundance.columns[:-1])]

# Save to CSV
viral_ce_relative_abundance.to_csv("/projects/luo_lab/Siders_data/results/tables/drep_viral_rel_abun_ce.csv", index=False)
viral_ce_relative_abundance

contig,samples,day0_DO_0_env_virus_control_000000000002,day0_DO_0_env_virus_control_000000000003,day0_DO_0_env_virus_control_000000000004,day0_DO_0_env_virus_control_000000000008,day0_DO_0_env_virus_control_000000000010,day0_DO_0_env_virus_control_000000000013,day0_DO_0_env_virus_control_000000000014,day0_DO_0_env_virus_control_000000000015,day0_DO_0_env_virus_control_000000000016,...,day7_DO_0_13C_virus_enriched_000000016806,day7_DO_0_13C_virus_enriched_000000016807,day7_DO_0_13C_virus_enriched_000000016808,day7_DO_0_13C_virus_enriched_000000016809,day7_DO_0_13C_virus_enriched_000000016810,day7_DO_0_13C_virus_enriched_000000016813,day7_DO_0_13C_virus_enriched_000000016815,day7_DO_0_13C_virus_enriched_000000016816,day7_DO_0_13C_virus_enriched_000000016817,day7_DO_0_13C_virus_enriched_000000016818
0,CLEAN_DAY0_DO_0_ENV_CELL_CONTROL_NONE,0.0,1.25543e-06,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.00000,0.0
1,DAY7_DO_0_12C_CELL_ENRICHED_10,0.0,0.00000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.00027,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00001,0.00004,0.0
2,DAY7_DO_0_12C_CELL_ENRICHED_6,0.0,0.00000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.00000,0.0
3,DAY7_DO_0_12C_CELL_ENRICHED_7,0.0,0.00000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.00000,0.0
4,DAY7_DO_0_12C_CELL_ENRICHED_8,0.0,1.17430e-04,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.00000,0.0
5,DAY7_DO_0_12C_CELL_ENRICHED_9,0.0,1.15665e-04,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.00003,0.0
6,DAY7_DO_0_13C_CELL_ENRICHED_5,0.0,0.00000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.00000,0.0
7,DAY7_DO_0_13C_CELL_ENRICHED_6,0.0,0.00000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.00000,0.0
8,DAY7_DO_0_13C_CELL_ENRICHED_7,0.0,0.00000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.00000,0.0
9,DAY7_DO_0_13C_CELL_ENRICHED_8,0.0,1.81532e-04,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.00000,0.0


Step 2: Use the 'org_dna_density' function to calculate the mean DNA density of each vOTUs:

In [41]:
viral_ce_relative_abundance_filter=viral_ce_relative_abundance[viral_ce_relative_abundance['samples'] != 'CLEAN_DAY0_DO_0_ENV_CELL_CONTROL_NONE']
viral_ce_relative_abundance_filter

# Calculate the sum of each column
column_sums = viral_ce_relative_abundance_filter.sum()

# Identify columns with a sum of zero
columns_to_drop = column_sums[column_sums == 0].index

# Drop the columns
viral_ce_relative_abundance_filter = viral_ce_relative_abundance_filter.drop(columns=columns_to_drop)


In [13]:
# Call in data frame of the gc content producted by seqkit:
vGC_content=pd.read_csv("/projects/luo_lab/Siders_data/results/tables/combined_vOTU_gc_results.txt", sep='\t',header=None, names=['organism','GC'])
vGC_content['GC']=vGC_content['GC']/100
vGC_content

,organism,GC
0,day0_DO_0_env_virus_control_000000000002,0.4238
1,day0_DO_0_env_virus_control_000000000003,0.4308
2,day0_DO_0_env_virus_control_000000000004,0.3146
3,day0_DO_0_env_virus_control_000000000008,0.5272
4,day0_DO_0_env_virus_control_000000000010,0.3953
...,...,...
31483,day7_DO_0_13C_virus_enriched_000000016813,0.3166
31484,day7_DO_0_13C_virus_enriched_000000016815,0.4090
31485,day7_DO_0_13C_virus_enriched_000000016816,0.3079
31486,day7_DO_0_13C_virus_enriched_000000016817,0.3582


In [43]:
viral_ce_relative_abundance_long=org_dna_denisity(viral_ce_relative_abundance_filter, cell_density_df, 'samples','organism','count', 'treatment','fraction')
viral_ce_relative_abundance_long

/tmp/ipykernel_1079306/1845785240.py:48: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped = joined_df.groupby(['organism', 'treatment']).apply(lambda x: (x['weighted']).sum())


treatment,organism,12C,13C
0,day0_DO_0_env_virus_control_000000000336,1.69567,NaN
1,day0_DO_0_env_virus_control_000000000405,NaN,1.69920
2,day0_DO_0_env_virus_control_000000000454,1.69553,NaN
3,day0_DO_0_env_virus_control_000000000574,1.69618,1.69500
4,day0_DO_0_env_virus_control_000000000575,1.69687,NaN
...,...,...,...
647,day7_DO_0_13C_virus_enriched_000000016717,NaN,1.70897
648,day7_DO_0_13C_virus_enriched_000000016725,1.69576,1.69629
649,day7_DO_0_13C_virus_enriched_000000016739,1.69506,1.69216
650,day7_DO_0_13C_virus_enriched_000000016770,1.70857,1.70888


In [44]:
viral_ce_relative_abundance_long=pd.merge(viral_ce_relative_abundance_long, vGC_content)
viral_ce_relative_abundance_long
#test_sorted = viral_ce_relative_abundance_long.sort_values(by='12C')
#test_sorted

,organism,12C,13C,GC
0,day0_DO_0_env_virus_control_000000000336,1.69567,NaN,0.3761
1,day0_DO_0_env_virus_control_000000000405,NaN,1.69920,0.4922
2,day0_DO_0_env_virus_control_000000000454,1.69553,NaN,0.3748
3,day0_DO_0_env_virus_control_000000000574,1.69618,1.69500,0.3790
4,day0_DO_0_env_virus_control_000000000575,1.69687,NaN,0.4416
...,...,...,...,...
647,day7_DO_0_13C_virus_enriched_000000016717,NaN,1.70897,0.4032
648,day7_DO_0_13C_virus_enriched_000000016725,1.69576,1.69629,0.4157
649,day7_DO_0_13C_virus_enriched_000000016739,1.69506,1.69216,0.3592
650,day7_DO_0_13C_virus_enriched_000000016770,1.70857,1.70888,0.5424


Step 3: Use the 'AtomFractionExcess' function to calculate the EAF of each vOTU in the cell enriched fraction and export to csv file to be used in R script for figure construction.

In [46]:
viralcell_atomic_fraction=AtomFractionExcess(viral_ce_relative_abundance_long).dropna()
viralcell_atomic_fraction.to_csv('/projects/luo_lab/Siders_data/results/tables/viral_eaf_ce.csv', index=False) 
viralcell_atomic_fraction


,organism,12C,13C,GC,shift,M_light,M_HeavyMax,M_Lab,EAF
3,day0_DO_0_env_virus_control_000000000574,1.69618,1.69500,0.3790,-0.00118,307.87898,317.66453,307.66538,-0.02159
7,day0_DO_0_env_virus_control_000000000649,1.69830,1.69781,0.4490,-0.00049,307.91370,317.66434,307.82503,-0.00899
9,day0_DO_0_env_virus_control_000000000681,1.69583,1.69681,0.3951,0.00098,307.88697,317.66449,308.06429,0.01793
11,day0_DO_0_env_virus_control_000000000689,1.70467,1.70766,0.5609,0.00299,307.96921,317.66403,308.50911,0.05507
16,day0_DO_0_env_virus_control_000000001151,1.70268,1.70516,0.5493,0.00248,307.96345,317.66407,308.41238,0.04576
...,...,...,...,...,...,...,...,...,...
646,day7_DO_0_13C_virus_enriched_000000016606,1.69554,1.69263,0.3497,-0.00291,307.86445,317.66461,307.33603,-0.05332
648,day7_DO_0_13C_virus_enriched_000000016725,1.69576,1.69629,0.4157,0.00052,307.89719,317.66443,307.99222,0.00962
649,day7_DO_0_13C_virus_enriched_000000016739,1.69506,1.69216,0.3592,-0.00290,307.86916,317.66458,307.34257,-0.05316
650,day7_DO_0_13C_virus_enriched_000000016770,1.70857,1.70888,0.5424,0.00031,307.96003,317.66408,308.01564,0.00567


In [17]:
#Add code here for viral taxonomy (either from kaiju or genomad)
virus_taxa=pd.read_csv("/projects/luo_lab/Siders_data/data/processed/taxonomy/genomad/representative_votus_summary/representative_votus_virus_summary.tsv", sep='\t')
virus_taxa=virus_taxa.rename(columns={'seq_name': 'organism'})
virus_taxa

,organism,length,topology,coordinates,n_genes,genetic_code,virus_score,fdr,n_hallmarks,marker_enrichment,taxonomy
0,day0_DO_0_env_virus_control_000000005303,12931,No terminal repeats,NaN,16,11,0.9838,NaN,3,26.4904,Viruses;Duplodnaviria;Heunggongvirae;Urovirico...
1,day0_DO_0_env_virus_control_000000006202,9365,No terminal repeats,NaN,13,11,0.9838,NaN,4,20.6363,Viruses;Duplodnaviria;Heunggongvirae;Urovirico...
2,day7_DO_0_13C_virus_enriched_000000001599,5837,No terminal repeats,NaN,13,11,0.9838,NaN,0,21.8027,Viruses;Duplodnaviria;Heunggongvirae;Urovirico...
3,day7_DO_0_12C_virus_enriched_000000000628,8301,No terminal repeats,NaN,14,11,0.9838,NaN,4,23.1047,Viruses;Duplodnaviria;Heunggongvirae;Urovirico...
4,day7_DO_0_13C_virus_enriched_000000011357,7085,No terminal repeats,NaN,13,11,0.9838,NaN,0,22.0750,Viruses;Duplodnaviria;Heunggongvirae;Urovirico...
...,...,...,...,...,...,...,...,...,...,...,...
30273,day0_DO_0_env_virus_control_000000015198,8023,No terminal repeats,NaN,14,11,0.7008,NaN,0,0.0000,Unclassified
30274,day0_DO_0_env_virus_control_000000008709,5328,No terminal repeats,NaN,6,11,0.7007,NaN,0,0.3264,Viruses;Duplodnaviria;Heunggongvirae;Urovirico...
30275,day7_DO_0_13C_virus_enriched_000000004812,12272,No terminal repeats,NaN,24,11,0.7005,NaN,0,0.0000,Unclassified
30276,day7_DO_0_13C_virus_enriched_000000010688,5783,No terminal repeats,NaN,8,11,0.7001,NaN,0,0.0000,Unclassified


In [18]:
# Select rows where the "length" column is at least 5000
virus_taxa_5k = virus_taxa[virus_taxa['length'] >= 5000]
virus_taxa_5k

,organism,length,topology,coordinates,n_genes,genetic_code,virus_score,fdr,n_hallmarks,marker_enrichment,taxonomy
0,day0_DO_0_env_virus_control_000000005303,12931,No terminal repeats,NaN,16,11,0.9838,NaN,3,26.4904,Viruses;Duplodnaviria;Heunggongvirae;Urovirico...
1,day0_DO_0_env_virus_control_000000006202,9365,No terminal repeats,NaN,13,11,0.9838,NaN,4,20.6363,Viruses;Duplodnaviria;Heunggongvirae;Urovirico...
2,day7_DO_0_13C_virus_enriched_000000001599,5837,No terminal repeats,NaN,13,11,0.9838,NaN,0,21.8027,Viruses;Duplodnaviria;Heunggongvirae;Urovirico...
3,day7_DO_0_12C_virus_enriched_000000000628,8301,No terminal repeats,NaN,14,11,0.9838,NaN,4,23.1047,Viruses;Duplodnaviria;Heunggongvirae;Urovirico...
4,day7_DO_0_13C_virus_enriched_000000011357,7085,No terminal repeats,NaN,13,11,0.9838,NaN,0,22.0750,Viruses;Duplodnaviria;Heunggongvirae;Urovirico...
...,...,...,...,...,...,...,...,...,...,...,...
30273,day0_DO_0_env_virus_control_000000015198,8023,No terminal repeats,NaN,14,11,0.7008,NaN,0,0.0000,Unclassified
30274,day0_DO_0_env_virus_control_000000008709,5328,No terminal repeats,NaN,6,11,0.7007,NaN,0,0.3264,Viruses;Duplodnaviria;Heunggongvirae;Urovirico...
30275,day7_DO_0_13C_virus_enriched_000000004812,12272,No terminal repeats,NaN,24,11,0.7005,NaN,0,0.0000,Unclassified
30276,day7_DO_0_13C_virus_enriched_000000010688,5783,No terminal repeats,NaN,8,11,0.7001,NaN,0,0.0000,Unclassified


In [21]:
virus_taxa_5k=virus_taxa_5k[['organism','taxonomy']]

# Extract the lowest taxonomic level
virus_taxa_5k['lowest_taxonomy'] = virus_taxa_5k['taxonomy'].str.split(';').str[-1]
virus_taxa_5k

/tmp/ipykernel_106587/1034390410.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  virus_taxa_5k['lowest_taxonomy'] = virus_taxa_5k['taxonomy'].str.split(';').str[-1]


,organism,taxonomy,lowest_taxonomy
0,day0_DO_0_env_virus_control_000000005303,Viruses;Duplodnaviria;Heunggongvirae;Urovirico...,Caudoviricetes
1,day0_DO_0_env_virus_control_000000006202,Viruses;Duplodnaviria;Heunggongvirae;Urovirico...,Caudoviricetes
2,day7_DO_0_13C_virus_enriched_000000001599,Viruses;Duplodnaviria;Heunggongvirae;Urovirico...,Caudoviricetes
3,day7_DO_0_12C_virus_enriched_000000000628,Viruses;Duplodnaviria;Heunggongvirae;Urovirico...,Caudoviricetes
4,day7_DO_0_13C_virus_enriched_000000011357,Viruses;Duplodnaviria;Heunggongvirae;Urovirico...,Caudoviricetes
...,...,...,...
30273,day0_DO_0_env_virus_control_000000015198,Unclassified,Unclassified
30274,day0_DO_0_env_virus_control_000000008709,Viruses;Duplodnaviria;Heunggongvirae;Urovirico...,Caudoviricetes
30275,day7_DO_0_13C_virus_enriched_000000004812,Unclassified,Unclassified
30276,day7_DO_0_13C_virus_enriched_000000010688,Unclassified,Unclassified


In [72]:
#Merge with taxa and density_values
viralcell_atomic_fraction = (
     viralcell_atomic_fraction
     .merge(virus_taxa_5k[['organism','lowest_taxonomy']], how="left")
 )
viralcell_atomic_fraction.to_csv('/projects/luo_lab/Siders_data/results/tables/viral_eaf_ce_taxa.csv', index=False) 
viralcell_atomic_fraction


,organism,12C,13C,GC,shift,M_light,M_HeavyMax,M_Lab,EAF,lowest_taxonomy
0,day0_DO_0_env_virus_control_000000000574,1.69618,1.69500,0.3790,-0.00118,307.87898,317.66453,307.66538,-0.02159,Caudoviricetes
1,day0_DO_0_env_virus_control_000000000649,1.69830,1.69781,0.4490,-0.00049,307.91370,317.66434,307.82503,-0.00899,NaN
2,day0_DO_0_env_virus_control_000000000681,1.69583,1.69681,0.3951,0.00098,307.88697,317.66449,308.06429,0.01793,Caudoviricetes
3,day0_DO_0_env_virus_control_000000000689,1.70467,1.70766,0.5609,0.00299,307.96921,317.66403,308.50911,0.05507,Caudoviricetes
4,day0_DO_0_env_virus_control_000000001151,1.70268,1.70516,0.5493,0.00248,307.96345,317.66407,308.41238,0.04576,Caudoviricetes
...,...,...,...,...,...,...,...,...,...,...
343,day7_DO_0_13C_virus_enriched_000000016606,1.69554,1.69263,0.3497,-0.00291,307.86445,317.66461,307.33603,-0.05332,Microviridae
344,day7_DO_0_13C_virus_enriched_000000016725,1.69576,1.69629,0.4157,0.00052,307.89719,317.66443,307.99222,0.00962,Caudoviricetes
345,day7_DO_0_13C_virus_enriched_000000016739,1.69506,1.69216,0.3592,-0.00290,307.86916,317.66458,307.34257,-0.05316,Caudoviricetes
346,day7_DO_0_13C_virus_enriched_000000016770,1.70857,1.70888,0.5424,0.00031,307.96003,317.66408,308.01564,0.00567,Caudoviricetes


# Part 3: Calculate the EAF of the vOTUs present in the virus enriched fraction 

Step 1: Call in required data frames for EAF calculation of the cell enriched fraction

In [3]:
# Define old and new names
voldnames = ["CLEAN_DAY7_DO_0_12C_VIRAL_10_12","CLEAN_DAY7_DO_0_12C_VIRAL_1_6",
              "CLEAN_DAY7_DO_0_12C_VIRAL_7","CLEAN_DAY7_DO_0_12C_VIRAL_8",
              "CLEAN_DAY7_DO_0_12C_VIRAL_9",
              
              "CLEAN_DAY7_DO_0_13C_VIRAL_10_12",
              "CLEAN_DAY7_DO_0_13C_VIRAL_1_6","CLEAN_DAY7_DO_0_13C_VIRAL_7",
              "CLEAN_DAY7_DO_0_13C_VIRAL_8","CLEAN_DAY7_DO_0_13C_VIRAL_9"]

vnewnames = ["DAY7_DO_0_12C_VIRAL_10","DAY7_DO_0_12C_VIRAL_6",
              "DAY7_DO_0_12C_VIRAL_7","DAY7_DO_0_12C_VIRAL_8",
              "DAY7_DO_0_12C_VIRAL_9",
              
              "DAY7_DO_0_13C_VIRAL_10",
              "DAY7_DO_0_13C_VIRAL_6","DAY7_DO_0_13C_VIRAL_7",
              "DAY7_DO_0_13C_VIRAL_8","DAY7_DO_0_13C_VIRAL_9"]


In [4]:
viral_ve_cov = pd.read_csv("/projects/luo_lab/Siders_data/data/processed/anvio/merged_profile_db/derep_viral_ve/derep_viral_ve-COVs.txt", sep="\t")  # Rename columns viral_ce_cov.rename(columns=dict(zip(oldnames, newnames)), inplace=True) viral_ce_cov.to_csv("/projects/luo_lab/Rogers_SidersViralAnalysis_XXXX_20XX/data/processed/anvio/merged_profile_db/derep_viral/viral_retentate_ce_cov_clean_names2.csv", index=False)  # Rename columns viral_ce_cov.rename(columns=dict(zip(oldnames, newnames)), inplace=True) #viral_ce_cov = viral_ce_cov.drop(columns=columns_to_remove) # Save to CSV #viral_ce_cov.to_csv("../../R/input_files/vcell_retentate_cov_clean_names.csv", index=False)  # Convert to DataFrame with contig as row index viral_ce_relative_abundance = viral_ce_cov.set_index("contig").transpose().reset_index()  # Normalize the data normalized_data = normalize(viral_ce_relative_abundance.drop(columns="index"), axis=1, norm='l1') viral_ce_relative_abundance = pd.DataFrame(normalized_data, columns=viral_ce_relative_abundance.columns[1:]) viral_ce_relative_abundance["samples"] = viral_ce_cov.columns[1:]  # Reorder columns to have samples first viral_ce_relative_abundance = viral_ce_relative_abundance[["samples"] + list(viral_ce_relative_abundance.columns[:-1])]  # Save to CSV viral_ce_relative_abundance.to_csv("/projects/luo_lab/Rogers_SidersViralAnalysis_XXXX_20XX/data/analysis/drep_vcell_rel_abun2.csv", index=False) viral_ve_relative_abundance = pd.read_csv("/projects/luo_lab/Rogers_SidersViralAnalysis_XXXX_20XX/data/processed/anvio/merged_profile_db/derep_viral/derep_viral-COVs.txt", sep="\t")  # Rename columns viral_ce_cov.rename(columns=dict(zip(oldnames, newnames)), inplace=True) viral_ce_cov.to_csv("/projects/luo_lab/Rogers_SidersViralAnalysis_XXXX_20XX/data/processed/anvio/merged_profile_db/derep_viral/viral_retentate_ce_cov_clean_names2.csv", index=False)  # Rename columns viral_ce_cov.rename(columns=dict(zip(oldnames, newnames)), inplace=True) #viral_ce_cov = viral_ce_cov.drop(columns=columns_to_remove) # Save to CSV #viral_ce_cov.to_csv("../../R/input_files/vcell_retentate_cov_clean_names.csv", index=False)  # Convert to DataFrame with contig as row index viral_ce_relative_abundance = viral_ce_cov.set_index("contig").transpose().reset_index()  # Normalize the data normalized_data = normalize(viral_ce_relative_abundance.drop(columns="index"), axis=1, norm='l1') viral_ce_relative_abundance = pd.DataFrame(normalized_data, columns=viral_ce_relative_abundance.columns[1:]) viral_ce_relative_abundance["samples"] = viral_ce_cov.columns[1:]  # Reorder columns to have samples first viral_ce_relative_abundance = viral_ce_relative_abundance[["samples"] + list(viral_ce_relative_abundance.columns[:-1])]  # Save to CSV viral_ce_relative_abundance.to_csv("/projects/luo_lab/Rogers_SidersViralAnalysis_XXXX_20XX/data/analysis/drep_vcell_rel_abun2.csv", index=False) viral_ve_relative_abundance = pd.read_csv("/projects/luo_lab/Rogers_SidersViralAnalysis_XXXX_20XX/data/processed/anvio/merged_profile_db/derep_viral/derep_viral-COVs.txt", sep="\t")
viral_ve_cov
# Rename columns
viral_ve_cov.rename(columns=dict(zip(voldnames, vnewnames)), inplace=True)
#viral_ve_cov.to_csv("", index=False)

In [6]:
# Convert to DataFrame with contig as row index
viral_ve_cov_transposed = viral_ve_cov.set_index("contig").transpose().reset_index()

# Normalize the data
normalized_data = normalize(viral_ve_cov_transposed.drop(columns="index"), axis=1, norm='l1')
viral_ve_relative_abundance = pd.DataFrame(normalized_data, columns=viral_ve_cov_transposed.columns[1:])
viral_ve_relative_abundance["samples"] = viral_ve_cov.columns[1:]

# Reorder columns to have samples first
viral_ve_relative_abundance = viral_ve_relative_abundance[["samples"] + list(viral_ve_relative_abundance.columns[:-1])]

# Save to CSV
viral_ve_relative_abundance.to_csv("/projects/luo_lab/Siders_data/results/tables/drep_ve_rel_abun.csv", index=False)
viral_ve_relative_abundance

contig,samples,day0_DO_0_env_virus_control_000000000002,day0_DO_0_env_virus_control_000000000003,day0_DO_0_env_virus_control_000000000004,day0_DO_0_env_virus_control_000000000008,day0_DO_0_env_virus_control_000000000010,day0_DO_0_env_virus_control_000000000013,day0_DO_0_env_virus_control_000000000014,day0_DO_0_env_virus_control_000000000015,day0_DO_0_env_virus_control_000000000016,...,day7_DO_0_13C_virus_enriched_000000016806,day7_DO_0_13C_virus_enriched_000000016807,day7_DO_0_13C_virus_enriched_000000016808,day7_DO_0_13C_virus_enriched_000000016809,day7_DO_0_13C_virus_enriched_000000016810,day7_DO_0_13C_virus_enriched_000000016813,day7_DO_0_13C_virus_enriched_000000016815,day7_DO_0_13C_virus_enriched_000000016816,day7_DO_0_13C_virus_enriched_000000016817,day7_DO_0_13C_virus_enriched_000000016818
0,CLEAN_DAY0_DO_0_ENV_VIRUS_CONTROL_NONE,8.352816e-06,5.366211e-05,0.000053,4.768499e-06,1.688471e-05,0.000008,1.638304e-05,0.000008,1.221921e-05,...,0.000008,3.340096e-06,7.827801e-06,2.417400e-06,0.000005,0.000001,3.060539e-06,3.692587e-06,2.326070e-05,0.000001
1,CLEAN_DAY7_DO_0_12C_VIRUS_ENRICHED_10_12,0.000000e+00,2.876467e-06,0.000108,0.000000e+00,5.874430e-06,0.000000,2.930393e-05,0.000000,1.651446e-05,...,0.000007,8.287784e-08,0.000000e+00,4.567401e-07,0.000003,0.000006,7.768225e-07,1.387161e-05,1.146088e-05,0.000002
2,CLEAN_DAY7_DO_0_12C_VIRUS_ENRICHED_1_6,0.000000e+00,2.638184e-05,0.000002,5.541557e-07,0.000000e+00,0.000000,3.554468e-07,0.000008,0.000000e+00,...,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000e+00,4.255649e-06,0.000016
3,CLEAN_DAY7_DO_0_12C_VIRUS_ENRICHED_7,0.000000e+00,2.236512e-05,0.000000,1.030882e-05,0.000000e+00,0.000000,0.000000e+00,0.000024,0.000000e+00,...,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000e+00,2.564879e-07,0.000014
4,CLEAN_DAY7_DO_0_12C_VIRUS_ENRICHED_8,4.638496e-06,1.607232e-05,0.000087,0.000000e+00,1.245934e-05,0.000000,2.643005e-05,0.000000,0.000000e+00,...,0.000029,4.145199e-06,6.694572e-07,7.575281e-06,0.000008,0.000035,2.986658e-08,0.000000e+00,4.943389e-05,0.000003
5,CLEAN_DAY7_DO_0_12C_VIRUS_ENRICHED_9,7.183588e-06,1.473982e-04,0.000012,0.000000e+00,1.998657e-05,0.000000,0.000000e+00,0.000015,0.000000e+00,...,0.000000,0.000000e+00,3.229521e-06,6.936768e-06,0.000002,0.000001,2.124402e-05,0.000000e+00,2.037501e-05,0.000016
6,CLEAN_DAY7_DO_0_13C_VIRUS_ENRICHED_10_12,0.000000e+00,1.357942e-06,0.000121,0.000000e+00,3.248740e-06,0.000000,3.220759e-05,0.000000,5.498667e-06,...,0.000016,5.175821e-06,0.000000e+00,0.000000e+00,0.000013,0.000021,0.000000e+00,1.221252e-05,6.159293e-05,0.000000
7,CLEAN_DAY7_DO_0_13C_VIRUS_ENRICHED_1_6,0.000000e+00,1.359996e-05,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00,...,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000e+00,7.760420e-07,0.000000
8,CLEAN_DAY7_DO_0_13C_VIRUS_ENRICHED_7,7.923722e-07,3.307415e-05,0.000004,1.683961e-05,3.194566e-07,0.000000,1.987772e-06,0.000021,3.511276e-07,...,0.000002,6.539598e-07,2.895152e-06,3.064958e-07,0.000001,0.000000,3.085242e-06,0.000000e+00,3.444562e-06,0.000013
9,CLEAN_DAY7_DO_0_13C_VIRUS_ENRICHED_8,6.999807e-06,9.416187e-05,0.000019,8.400118e-07,1.273613e-05,0.000000,5.351773e-06,0.000012,1.591811e-06,...,0.000015,5.845163e-06,1.427145e-05,1.362235e-05,0.000011,0.000012,1.316732e-05,9.374288e-07,3.885116e-05,0.000014


Step 2: Use the 'org_dna_density' function to calculate the mean DNA density of each vOTUs:

In [7]:
viral_ve_relative_abundance_filter=viral_ve_relative_abundance[viral_ve_relative_abundance['samples'] != 'CLEAN_DAY0_DO_0_ENV_VIRUS_CONTROL_NONE']
viral_ve_relative_abundance_filter

# Calculate the sum of each column
column_sums = viral_ve_relative_abundance_filter.sum()

# Identify columns with a sum of zero
columns_to_drop = column_sums[column_sums == 0].index

# Drop the columns
viral_ve_relative_abundance_filter = viral_ve_relative_abundance_filter.drop(columns=columns_to_drop)


Step 3: Use the 'AtomFractionExcess' function to calculate the EAF of each vOTU in the viral enriched fraction and export to csv file to be used in R script for figure construction.

In [8]:
# Create the cell_density_df DataFrame
viral_density_df = pd.DataFrame({
    "fraction": ["6","7","8","9","10", "6","7","8","9","10"],
    "treatment": ["12C", "12C", "12C", "12C", "12C", "13C", "13C", "13C", "13C", "13C"],
    "density": [1.71001104,1.70454724,1.69908344,1.69361964,1.69034136,
                1.7111038,1.70454724,1.69799068,1.69252688,1.68706308],

    "qpcr_ratio": [0.166359643,0.390011325,0.62654109,1,0.531697631,
                   0.170752811,0.455733261,0.92387403,1,0.341295116],
    "filtrate_type": ["viral fraction"] * 10
})

# Remove rows where fraction is 12 and drop filtrate_type column
viral_density_df = viral_density_df[viral_density_df["fraction"] != "12"].drop(columns=["filtrate_type"])

# Save to CSV
viral_density_df.to_csv("/projects/luo_lab/Siders_data/results/tables/viral_density_table.csv", index=False)
viral_density_df

,fraction,treatment,density,qpcr_ratio
0,6,12C,1.710011,0.166360
1,7,12C,1.704547,0.390011
2,8,12C,1.699083,0.626541
3,9,12C,1.693620,1.000000
4,10,12C,1.690341,0.531698
5,6,13C,1.711104,0.170753
6,7,13C,1.704547,0.455733
7,8,13C,1.697991,0.923874
8,9,13C,1.692527,1.000000
9,10,13C,1.687063,0.341295


In [11]:
viral_ve_relative_abundance_long=org_dna_denisity(viral_ve_relative_abundance_filter, viral_density_df, 'samples','organism','count', 'treatment','fraction')
viral_ve_relative_abundance_long

/tmp/ipykernel_106587/1845785240.py:48: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped = joined_df.groupby(['organism', 'treatment']).apply(lambda x: (x['weighted']).sum())


treatment,organism,12C,13C
0,day0_DO_0_env_virus_control_000000000003,1.694923,1.699176
1,day0_DO_0_env_virus_control_000000000004,1.698192,1.693390
2,day0_DO_0_env_virus_control_000000000010,1.695154,1.697046
3,day0_DO_0_env_virus_control_000000000014,1.699122,1.693492
4,day0_DO_0_env_virus_control_000000000015,1.698424,NaN
...,...,...,...
25338,day7_DO_0_13C_virus_enriched_000000016813,1.698799,1.694594
25339,day7_DO_0_13C_virus_enriched_000000016815,1.693624,1.698461
25340,day7_DO_0_13C_virus_enriched_000000016816,NaN,1.692814
25341,day7_DO_0_13C_virus_enriched_000000016817,1.697108,1.695503


In [14]:
viral_ve_relative_abundance_long=pd.merge(viral_ve_relative_abundance_long, vGC_content)
viral_ve_relative_abundance_long

,organism,12C,13C,GC
0,day0_DO_0_env_virus_control_000000000003,1.694923,1.699176,0.4308
1,day0_DO_0_env_virus_control_000000000004,1.698192,1.693390,0.3146
2,day0_DO_0_env_virus_control_000000000010,1.695154,1.697046,0.3953
3,day0_DO_0_env_virus_control_000000000014,1.699122,1.693492,0.3130
4,day0_DO_0_env_virus_control_000000000015,1.698424,NaN,0.5054
...,...,...,...,...
25338,day7_DO_0_13C_virus_enriched_000000016813,1.698799,1.694594,0.3166
25339,day7_DO_0_13C_virus_enriched_000000016815,1.693624,1.698461,0.4090
25340,day7_DO_0_13C_virus_enriched_000000016816,NaN,1.692814,0.3079
25341,day7_DO_0_13C_virus_enriched_000000016817,1.697108,1.695503,0.3582


In [15]:
viral_ve_atomic_fraction=AtomFractionExcess(viral_ve_relative_abundance_long).dropna()
viral_ve_atomic_fraction.to_csv('/projects/luo_lab/Siders_data/results/tables/viral_eaf_ve.csv', index=False) 
viral_ve_atomic_fraction


,organism,12C,13C,GC,shift,M_light,M_HeavyMax,M_Lab,EAF
0,day0_DO_0_env_virus_control_000000000003,1.694923,1.699176,0.4308,0.004253,307.904677,317.664389,308.677300,0.078285
1,day0_DO_0_env_virus_control_000000000004,1.698192,1.693390,0.3146,-0.004801,307.847042,317.664706,306.976640,-0.087671
2,day0_DO_0_env_virus_control_000000000010,1.695154,1.697046,0.3953,0.001892,307.887069,317.664486,308.230665,0.034751
3,day0_DO_0_env_virus_control_000000000014,1.699122,1.693492,0.3130,-0.005630,307.846248,317.664710,306.826199,-0.102736
6,day0_DO_0_env_virus_control_000000000017,1.695160,1.699436,0.4616,0.004276,307.919954,317.664305,308.696671,0.078824
...,...,...,...,...,...,...,...,...,...
25336,day7_DO_0_13C_virus_enriched_000000016809,1.695839,1.697801,0.3767,0.001961,307.877843,317.664536,308.233948,0.035982
25337,day7_DO_0_13C_virus_enriched_000000016810,1.697556,1.695477,0.3584,-0.002080,307.868766,317.664586,307.491622,-0.038073
25338,day7_DO_0_13C_virus_enriched_000000016813,1.698799,1.694594,0.3166,-0.004205,307.848034,317.664700,307.086101,-0.076754
25339,day7_DO_0_13C_virus_enriched_000000016815,1.693624,1.698461,0.4090,0.004837,307.893864,317.664448,308.773121,0.088990


In [23]:
#Merge with taxa and density_values
viral_ve_atomic_fraction = (
     viral_ve_atomic_fraction
     .merge(virus_taxa_5k[['organism','lowest_taxonomy']], how="left")
 )
viral_ve_atomic_fraction.to_csv('/projects/luo_lab/Siders_data/results/tables/viral_eaf_ve_taxa.csv', index=False) 
viral_ve_atomic_fraction

,organism,12C,13C,GC,shift,M_light,M_HeavyMax,M_Lab,EAF,lowest_taxonomy
0,day0_DO_0_env_virus_control_000000000003,1.694923,1.699176,0.4308,0.004253,307.904677,317.664389,308.677300,0.078285,Caudoviricetes
1,day0_DO_0_env_virus_control_000000000004,1.698192,1.693390,0.3146,-0.004801,307.847042,317.664706,306.976640,-0.087671,Caudoviricetes
2,day0_DO_0_env_virus_control_000000000010,1.695154,1.697046,0.3953,0.001892,307.887069,317.664486,308.230665,0.034751,Caudoviricetes
3,day0_DO_0_env_virus_control_000000000014,1.699122,1.693492,0.3130,-0.005630,307.846248,317.664710,306.826199,-0.102736,Caudoviricetes
4,day0_DO_0_env_virus_control_000000000017,1.695160,1.699436,0.4616,0.004276,307.919954,317.664305,308.696671,0.078824,Caudoviricetes
...,...,...,...,...,...,...,...,...,...,...
14776,day7_DO_0_13C_virus_enriched_000000016809,1.695839,1.697801,0.3767,0.001961,307.877843,317.664536,308.233948,0.035982,Kyanoviridae
14777,day7_DO_0_13C_virus_enriched_000000016810,1.697556,1.695477,0.3584,-0.002080,307.868766,317.664586,307.491622,-0.038073,Kyanoviridae
14778,day7_DO_0_13C_virus_enriched_000000016813,1.698799,1.694594,0.3166,-0.004205,307.848034,317.664700,307.086101,-0.076754,Crassvirales
14779,day7_DO_0_13C_virus_enriched_000000016815,1.693624,1.698461,0.4090,0.004837,307.893864,317.664448,308.773121,0.088990,Unclassified
